In [1]:
import sys, time
from nltk import tokenize
from nltk.grammar import toy_pcfg1
from nltk.parse import pchart
from nltk.parse import ViterbiParser

demos = [('I saw John with my telescope', toy_pcfg1)]
sent, grammar = demos[0]

# Tokenize the sentence.
tokens = sent.split()

# Define a list of parsers.  We'll use all parsers.
parsers = [
ViterbiParser(grammar),
pchart.InsideChartParser(grammar),
pchart.RandomChartParser(grammar),
pchart.UnsortedChartParser(grammar),
pchart.LongestChartParser(grammar),
pchart.InsideChartParser(grammar, beam_size = len(tokens)+1)
]

In [2]:
# Run the parsers on the tokenized sentence.
from functools import reduce
times = []
average_p = []
num_parses = []
all_parses = {}
for parser in parsers:
    print('\ns: %s\nparser: %s\ngrammar: %s' % (sent,parser,grammar))
    parser.trace(3)
    t = time.time()
    parses = parser.parse_all(tokens)
    times.append(time.time()-t)
    if parses: 
        lp = len(parses)
        p = reduce(lambda a,b:a+b.prob(), parses, 0.0)
    else: 
        p = 0
    average_p.append(p)
    num_parses.append(len(parses))
    for p in parses: 
        all_parses[p.freeze()] = 1

# Print summary statistics
print()
print('-------------------------+------------------------------------------')
print('   Parser           Beam | Time (secs)   # Parses   Average P(parse)')
print('-------------------------+------------------------------------------')
for i in range(len(parsers)):
    print('%19s %4d |%11.4f%11d%19.14f' % (parsers[i].__class__.__name__,
      getattr(parsers[0], "beam_size", 0),
      times[i], 
      num_parses[i], 
      average_p[i]))
parses = all_parses.keys()
if parses: 
    p = reduce(lambda a,b:a+b.prob(), parses, 0)/len(parses)
else: 
    p = 0
print('-------------------------+------------------------------------------')
print('%19s      |%11s%11d%19.14f' % ('(All Parses)', 'n/a', len(parses), p))
print()

for parse in parses:
    print(parse)

[-] . . . .| [1:2] 'saw'                        [1.0]
  |. . [-] . . .| [2:3] 'John'                       [1.0]
  |. . . [-] . .| [3:4] 'with'                       [1.0]
  |. . . . [-] .| [4:5] 'my'                         [1.0]
  |. . . . . [-]| [5:6] 'telescope'                  [1.0]
  |. . . . . [-]| [5:6] 'telescope'                  [1.0]
  |. . . . [-] .| [4:5] 'my'                         [1.0]
  |. . . [-] . .| [3:4] 'with'                       [1.0]
  |. . [-] . . .| [2:3] 'John'                       [1.0]
  |. [-] . . . .| [1:2] 'saw'                        [1.0]
  |[-] . . . . .| [0:1] 'I'                          [1.0]
  |. [-] . . . .| [1:2] V  -> 'saw' *                [0.65]
  |. > . . . . .| [1:1] VP -> * V NP                 [0.7]
  |. > . . . . .| [1:1] V  -> * 'saw'                [0.65]
  |. . . [-] . .| [3:4] P  -> 'with' *               [0.61]
  |. . . > . . .| [3:3] PP -> * P NP                 [1.0]
  |. . . [-> . .| [3:4] PP -> P * NP                 [0.61